In [4]:
import os
import cv2
import time
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from scipy import misc
from imutils import paths
from Detection import Detection
from Recognition import Recognition
import align_dataset
import preprocess
import pandas as pd
import gen_classifier
import classify_image

In [5]:
#Input and Output paths
train_raw_path = "/home/santoshv/RT_facenet/Dataset/AI_team_augmented/"
train_aligned_path = "/home/santoshv/RT_facenet/Dataset/AI_team_augmented/"
output_classifier_path = "../util/EximiusAI_1.pkl"
test_image_input_path = "/home/santoshv/facenet/data/images/test_raw/ravikiran/Capture+_2019-02-19-16-37-01.png"
augmentation = False
#Not yet Implemented.
if augmentation == True:
    augmentation_folder_path = "../Dataset/augmentation_folder/"

In [6]:
#Path for the detection and recognition pb files
Detection_model_path = "../Models/Detection_mtcnn.pb"
Recognition_model_path = "../Models/Recognition_facenet.pb"

In [7]:
#Instances of detection and recognition are being created.
#Instances are created to avoid loading the graphs and sessions again and again for every frame.
detection = Detection(Detection_model_path)
recognition = Recognition(Recognition_model_path)


Detection Model Graph Initialized
Recognition Model Graph Initialized


In [8]:
#creating the identity directories inside the output folder
#cropping faces using MTCNN face detection model from the model path 
#writing the respective faces in the directories created
#this function needs the paths of raw training images folder, output aligned folder and detection model path
#We suggest to manually remove noise from the aligned folder as MTCNN can detect some false positives which will
#effect recognition performance.Making sure we send right data to the model will leave us with good accuracy.
align_dataset.align_data(train_raw_path,train_aligned_path,detection)

In [9]:
#Getting the features of the faces stored at train_aligned_path in the form of 512-D embeddings
#We are using the FaceNet Inception-Resnet architecture to generate the embeddings which uses softmax loss
#The model is not the same that was mentioned in facenet paper, as it has different neural structure and loss
#Generated embeddings and the labels are given to a support vector classfier 
#The trained model is stored in the file path given to the function
gen_classifier.rec_gen_classifier(train_aligned_path , recognition ,output_classifier_path)

/home/santoshv/.local/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=5,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False) ['Manjunath', 'Santosh', 'Nimisha', 'Rohit', 'Sandipan', 'SaiKrishna', 'Sudheep', 'Soumallya', 'Ravi', 'Ayushee', 'Ramakrishna']


In [11]:
#This function takes an input image
#crops the faces out
#gets the embedding vector of each face
#loads the SVC model from the classifier pickle file
#predicts the identity 
classify_image.classify_image(output_classifier_path , detection , recognition ,test_image_input_path )

['Ravi']     [[1.54047751e-03 3.02932508e-04 8.77148801e-04 1.12708062e-03
  9.23990105e-01 1.70262254e-02 1.05274972e-02 2.76773087e-02
  5.13180340e-03 1.16344876e-02 1.64933490e-04]]
0.923990104666959
4


In [8]:
with open(output_classifier_path, 'rb') as infile:
        model ,class_names = pickle.load(infile)